# Sentinel-1 Preprocessing Workflow

In [1]:
import os
from pathlib import Path
from datetime import datetime

import gc
gc.enable()

# snappy imports
import snappy
from snappy import ProductIO, GPF

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

In [2]:
# scihub credentials
SCIHUB_UN = %env SCIHUB_UN
SCIHUB_PW = %env SCIHUB_WP

# request parameters - more TBD
REQUEST_AREA = "POLYGON ((6.2567138671875 51.57536261053028, 6.7160797119140625 51.57536261053028, 6.7160797119140625 51.78865666323309, 6.2567138671875 51.78865666323309, 6.2567138671875 51.57536261053028))"

# output directories 
TMP_OUTPUT_DIR = "./output/tmp/"
FNL_OUTPUT_DIR = "./output/"

In [3]:
Path(TMP_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
Path(FNL_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

## Download of Sentinel-1 Data

In [ ]:
# request API
api = SentinelAPI(SCIHUB_UN, SCIHUB_PW)
products = api.query(
    area=AOI_WKT, 
    producttype='GRD',
    date=(date(2020, 1, 8), date(2020,1,9)),
    platformname = "Sentinel-1")

# just get the first item
first_product = products.popitem()
s1_raw_filename = first_product[1]['title'] + ".zip"

# download file (doesn't get downloaded if already exists)
downloaded_file = api.download(first_product[0], directory_path=FNL_OUTPUT_DIR)
downloaded_file

In [ ]:
import folium
from shapely import wkt, geometry

dataset_footprint = wkt.loads(downloaded_file['footprint'])
aoi_footprint = wkt.loads(AOI_WKT)

m = folium.Map(zoom_starts=3)
m.fit_bounds((
    (dataset_footprint.bounds[1], dataset_footprint.bounds[0]), 
    (dataset_footprint.bounds[3], dataset_footprint.bounds[2])))
folium.GeoJson(dataset_footprint).add_to(m)
folium.GeoJson(aoi_footprint).add_to(m)
m